# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `warehouse_and_retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per supplier.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [2]:
# your code here
import pandas as pd

data = pd.read_csv('Warehouse_and_Retail_Sales.csv')
data

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.00,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.00,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.00,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.00,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.00,4.0,0.0
...,...,...,...,...,...,...,...,...,...
128350,2018,2,ANHEUSER BUSCH INC,9997,HOEGAARDEN 4/6NR - 12OZ,BEER,66.46,59.0,212.0
128351,2018,2,COASTAL BREWING COMPANY LLC,99970,DOMINION OAK BARREL STOUT 4/6 NR - 12OZ,BEER,9.08,7.0,35.0
128352,2018,2,BOSTON BEER CORPORATION,99988,SAM ADAMS COLD SNAP 1/6 KG,KEGS,0.00,0.0,32.0
128353,2018,2,NaN,BC,BEER CREDIT,REF,0.00,0.0,-35.0


In [3]:
# searching for null data points
null_cols = data.isnull().sum()
null = null_cols[null_cols > 0]
print(null)

SUPPLIER     24
ITEM TYPE     1
dtype: int64


In [5]:
# I indentify that null supplier values represent credit purchases.
# I will drop credit purchases to have a clean table with only actual and present values.

null_supplier = data.loc[data['SUPPLIER'].isnull()].index
data = data.drop(null_supplier)

#checking if rows have been succesfully dropped
null_cols = data.isnull().sum()
null = null_cols[null_cols > 0]
print(null)

ITEM TYPE    1
dtype: int64


In [7]:
# checking data types
data.dtypes

# they all seem correct and appropiate

YEAR                  int64
MONTH                 int64
SUPPLIER             object
ITEM CODE            object
ITEM DESCRIPTION     object
ITEM TYPE            object
RETAIL SALES        float64
RETAIL TRANSFERS    float64
WAREHOUSE SALES     float64
dtype: object

In [15]:
# 1. A table for the clean data:
data

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.00,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.00,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.00,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.00,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.00,4.0,0.0
...,...,...,...,...,...,...,...,...,...
128348,2018,2,LEGENDS LTD,99753,DUTCHESS DE BOURGOGNE NR - 750ML,BEER,0.00,0.0,2.0
128349,2018,2,COASTAL BREWING COMPANY LLC,99813,DOMINION OAK BARREL STOUT 1/2K,KEGS,0.00,0.0,2.0
128350,2018,2,ANHEUSER BUSCH INC,9997,HOEGAARDEN 4/6NR - 12OZ,BEER,66.46,59.0,212.0
128351,2018,2,COASTAL BREWING COMPANY LLC,99970,DOMINION OAK BARREL STOUT 4/6 NR - 12OZ,BEER,9.08,7.0,35.0


In [26]:
# grouping by supplier and executing the aggregate function
agg_supplier = data.groupby(['SUPPLIER'])
agg_supplier = agg_supplier['RETAIL SALES','RETAIL TRANSFERS','WAREHOUSE SALES'].agg(sum)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [27]:
# 2. A table for the aggregate per supplier:
agg_supplier

,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
SUPPLIER,,,
8 VINI INC,2.78,2.00,1.00
A HARDY USA LTD,0.40,0.00,0.00
A I G WINE & SPIRITS,12.52,5.92,134.00
A VINTNERS SELECTIONS,8640.57,8361.10,29776.67
A&E INC,11.52,2.00,0.00
...,...,...,...
WINEBOW INC,1.24,-1.58,0.00
YOUNG WON TRADING INC,1058.65,1047.40,2528.90
YUENGLING BREWERY,9628.35,10851.17,53805.32


In [28]:
# grouping by item and executing the aggregate function
agg_item = data.groupby(['ITEM CODE'])
agg_item = agg_item['RETAIL SALES','RETAIL TRANSFERS','WAREHOUSE SALES'].agg(sum)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
# 3. A table for the aggregate per item:
agg_item

,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
ITEM CODE,,,
100003,0.00,0.00,1.00
100007,0.00,0.00,1.00
100008,0.00,0.00,1.00
100009,0.00,0.00,12.00
100011,0.00,0.00,3.00
...,...,...,...
9989,1.00,1.00,9.00
9997,791.60,907.92,2384.05
99970,118.24,118.00,456.00


In [ ]:
# NOTES

In [19]:
#filtered = df[(df['ITEM DESCRIPTION']=='BEER CREDIT') & 
                #(df['ITEM DESCRIPTION']=='WINE CREDIT')]

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES


In [16]:
#df.query('str.contains("credit")', engine='python')
#df[df.summary.str.contains('crdit', case=False)]

In [28]:
data.aggregate({"SUPPLIER":['max', 'min'], 
              "ITEM CODE":['max', 'min'], 
              "ITEM TYPE":['max', 'min'],
              "RETAIL SALES":['max', 'min', 'sum', 'avg'], 
              "RETAIL TRANSFERS":['max', 'min', 'sum', 'avg'], 
              "WAREHOUSE SALES":['max', 'min', 'sum', 'avg']})

ValueError: no results

In [29]:
avg_mpg = data.groupby('SUPPLIER', as_index=False).agg({'Combined MPG':'mean'})
avg_mpg.columns = ['SUPPLIER', 'Avg_MPG']
avg_mpg.head()

SpecificationError: nested renamer is not supported

In [32]:
avg_mpg = data.groupby('SUPPLIER', as_index=False)
avg_mpg

In [34]:
data.agg({'Combined MPG':'mean'})

SpecificationError: nested renamer is not supported